In [1]:
import json
import openmeteo_requests
import requests_cache
import pandas as pd
from tqdm import tqdm
from retry_requests import retry

In [2]:
# Загрузка JSON с городами и их координатами
with open("data/source/cities_data.json", "r", encoding="utf-8") as f:
    cities_data = json.load(f)

In [3]:
print(len(cities_data)) # Кол-во городов в справочнике
cities_data[-1]

358


{'name': 'Филлах', 'country': 'Австрия', 'latitude': 46.61, 'longitude': 13.86}

In [7]:
# Open-Meteo — это API погоды с открытым исходным кодом, предлагающий бесплатный доступ для некоммерческого использования
url = "https://air-quality-api.open-meteo.com/v1/air-quality"

# Для сбора данных
data_air_quality = []

In [9]:
# Настройка сессии для работы с API
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600) # кэширование для хранения ответов на запросы
retry_session = retry(cache_session, retries = 3, backoff_factor = 2) # повторная попытка в случае ошибки
openmeteo = openmeteo_requests.Client(session = retry_session)

In [11]:
def parser_air_quality(cities_data):
    # Извлечение данных из первой половины JSON
    cyties     = [city["name"]      for city in cities_data]
    countries  = [city["country"]   for city in cities_data]
    latitudes  = [city["latitude"]  for city in cities_data]
    longitudes = [city["longitude"] for city in cities_data]

    # Параметры для запроса (одинаковые для всех городов)
    params = {
        "hourly": [
            "pm10", "pm2_5", "carbon_monoxide", "carbon_dioxide",
            "nitrogen_dioxide", "sulphur_dioxide", "ozone",
            "aerosol_optical_depth", "dust", "uv_index",
            "uv_index_clear_sky", "ammonia", "methane",
            "alder_pollen", "birch_pollen", "grass_pollen",
            "mugwort_pollen", "olive_pollen", "ragweed_pollen"
        ],
        "latitude": latitudes,
        "longitude": longitudes,
        "start_date": "2020-01-01",
        "end_date": "2024-12-31"
    }

    responses = openmeteo.weather_api(url, params=params)

    # Обработка данных для каждого местоположения + индикатор отслеживания
    for i, response in tqdm(enumerate(responses), desc="Обработка городов"):
        city      = cyties[i]
        country   = countries[i]
        latitude  = latitudes[i]
        longitude = longitudes[i]

        # Извлечение почасовых данных
        hourly = response.Hourly()
        timestamps = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )

        data_dict = {
            "date":                  timestamps,
            "country":               country,
            "city":                  city,
            "latitude":              latitude,
            "longitude":             longitude,
            "pm10":                  hourly.Variables(0).ValuesAsNumpy(),
            "pm2_5":                 hourly.Variables(1).ValuesAsNumpy(),
            "carbon_monoxide":       hourly.Variables(2).ValuesAsNumpy(),
            "carbon_dioxide":        hourly.Variables(3).ValuesAsNumpy(),
            "nitrogen_dioxide":      hourly.Variables(4).ValuesAsNumpy(),
            "sulphur_dioxide":       hourly.Variables(5).ValuesAsNumpy(),
            "ozone":                 hourly.Variables(6).ValuesAsNumpy(),
            "aerosol_optical_depth": hourly.Variables(7).ValuesAsNumpy(),
            "dust":                  hourly.Variables(8).ValuesAsNumpy(),
            "uv_index":              hourly.Variables(9).ValuesAsNumpy(),
            "uv_index_clear_sky":    hourly.Variables(10).ValuesAsNumpy(),
            "ammonia":               hourly.Variables(11).ValuesAsNumpy(),
            "methane":               hourly.Variables(12).ValuesAsNumpy(),
            "alder_pollen":          hourly.Variables(13).ValuesAsNumpy(),
            "birch_pollen":          hourly.Variables(14).ValuesAsNumpy(),
            "grass_pollen":          hourly.Variables(15).ValuesAsNumpy(),
            "mugwort_pollen":        hourly.Variables(16).ValuesAsNumpy(),
            "olive_pollen":          hourly.Variables(17).ValuesAsNumpy(),
            "ragweed_pollen":        hourly.Variables(18).ValuesAsNumpy()
        }

        df = pd.DataFrame(data_dict)
        data_air_quality.append(df)

In [13]:
# Разбиение данных json на две части, так как клиент Open-Meteo API не позволяет делать все запросы за раз (ограничения сайта)
half = len(cities_data) // 2
half

179

In [43]:
parser_air_quality(cities_data[:half])

Обработка городов: 179it [00:00, 298.27it/s]


In [17]:
# Второй запуск через час, когда пройдет ограничение на запросы
parser_air_quality(cities_data[half:])

Обработка городов: 179it [00:00, 194.19it/s]


In [19]:
# Создание итогового DataFrame
final_dataframe = pd.concat([pd.DataFrame(data) for data in data_air_quality], ignore_index=True)
final_dataframe.head()

,date,country,city,latitude,longitude,pm10,pm2_5,carbon_monoxide,carbon_dioxide,nitrogen_dioxide,...,uv_index,uv_index_clear_sky,ammonia,methane,alder_pollen,birch_pollen,grass_pollen,mugwort_pollen,olive_pollen,ragweed_pollen
0,2020-01-01 00:00:00+00:00,Германия,Штутгарт,48.78,9.18,43.599998,33.700001,377.0,NaN,43.299999,...,NaN,NaN,2.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-01 01:00:00+00:00,Германия,Штутгарт,48.78,9.18,52.299999,33.599998,362.0,NaN,43.299999,...,NaN,NaN,2.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01 02:00:00+00:00,Германия,Штутгарт,48.78,9.18,53.599998,33.099998,383.0,NaN,41.500000,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-01 03:00:00+00:00,Германия,Штутгарт,48.78,9.18,53.900002,35.299999,362.0,NaN,36.500000,...,NaN,NaN,1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-01 04:00:00+00:00,Германия,Штутгарт,48.78,9.18,53.599998,35.299999,352.0,NaN,31.400000,...,NaN,NaN,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15697584 entries, 0 to 15697583
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   date                   object 
 1   country                object 
 2   city                   object 
 3   latitude               float64
 4   longitude              float64
 5   pm10                   float64
 6   pm2_5                  float64
 7   carbon_monoxide        float64
 8   carbon_dioxide         float64
 9   nitrogen_dioxide       float64
 10  sulphur_dioxide        float64
 11  ozone                  float64
 12  aerosol_optical_depth  float64
 13  dust                   float64
 14  uv_index               float64
 15  uv_index_clear_sky     float64
 16  ammonia                float64
 17  methane                float64
 18  alder_pollen           float64
 19  birch_pollen           float64
 20  grass_pollen           float64
 21  mugwort_pollen         float64
 22  olive_pollen    

In [29]:
# Проверка, что нашлись все данные
358*(2*366*24 + 3*365*24) == final_dataframe.shape[0]

True

In [31]:
final_dataframe.to_csv('air_quality.csv', index=False)

print("Данные были сохранены в файл 'air_quality.csv'")

Данные были сохранены в файл 'air_quality.csv'
